# Rollout Datasets

Each model trained on non-deterministic A* search is evaluated by prompting the model with a test prompt and then randomly generating 64 response token sequences.
This dataset can be downloaded [here](https://dl.fbaipublicfiles.com/searchformer/rolloutDB.gz) and can be imported with [`mongorestore`](https://www.mongodb.com/docs/database-tools/mongorestore/).
The archive can be downloaded and imported by running

```
wget https://dl.fbaipublicfiles.com/searchformer/rolloutDB.gz
mongorestore --gzip --archive=rolloutDB.gz
```

After importing, the included rollout datasets can be listed with the following code.

In [1]:
import sys; sys.path.append("..")

import logging
from searchformer.rollout import RolloutDataStore


logging.basicConfig(
    level=logging.DEBUG,
    format="%(levelname)s - %(asctime)s - %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

datastore = RolloutDataStore()
datastore.list_all().sort_values(by=["checkpoint_id"])

INFO - 2024-04-26 13:56:23 - root - Connecting to mongodb://localhost:27017/mongo


,_id,checkpoint_id,dataset_name,sampler_name,rollout_len,rollout_repeats,prefix_len,min_reasoning_len,max_reasoning_len
154,65a2c15d31734efabad7003c,maze-sweep-rep-nondet-large-0,maze.10-by-10-nondeterministic.simple,probability,10000,64,0,-1,-1
186,65a2c1633ceb62baa214dbc9,maze-sweep-rep-nondet-large-1,maze.20-by-20-nondeterministic.simple,probability,10000,64,0,-1,-1
222,65a693bec546ebb8f2c67e30,maze-sweep-rep-nondet-large-10,maze.20-by-20-nondeterministic.simple,probability,10000,64,0,-1,-1
182,65a2c1630be0d0d343758179,maze-sweep-rep-nondet-large-100,maze.20-by-20-nondeterministic.simple,probability,10000,64,0,-1,-1
216,65a693be253ea5853a6b8e2e,maze-sweep-rep-nondet-large-101,maze.30-by-30-nondeterministic.simple,probability,10000,64,0,-1,-1
...,...,...,...,...,...,...,...,...,...
361,65ca8ea212eb3ce0932a107c,sokoban-7722-m-trace-plan-100k-2-step-2,sokoban.7-by-7-walls-2-boxes-2.with-box-40k,probability,11000,64,0,0,10000
364,65cbb28a92054ef71f553bae,sokoban-7722-m-trace-plan-100k-2-step-3,sokoban.7-by-7-walls-2-boxes-2.with-box-40k,probability,11000,64,0,0,10000
347,65b6bc24ff07df4ca9191ab8,sokoban-7722-xl-plan-only-100k-0,sokoban.7-by-7-walls-2-boxes-2.with-box-40k,probability,2000,64,0,0,10000
348,65b6bc2552b35c10a841e311,sokoban-7722-xl-plan-only-100k-1,sokoban.7-by-7-walls-2-boxes-2.with-box-40k,probability,2000,64,0,0,10000


## Loading generated response sequences

A rollout dataset can be loaded with its corresponding id (`_id` column in the dataframe outputted above).
Then, one can iterate over the generated response sequences to load each stored response.

In [2]:
dataset = datastore.load_by_id("65a1dfa305a44a3151e7849c")
rollout = next(iter(dataset.rollout_test_it()))

print("Dataset parameters:")
for k, v in dataset.params.to_doc().items():
    print(f"\t{k}: {v}")

INFO - 2024-04-26 13:56:23 - root - Connecting to mongodb://localhost:27017/mongo
DEBUG - 2024-04-26 13:56:23 - root - Loading 100 ids.


Dataset parameters:
	checkpoint_id: maze-sweep-rep-nondet-small-0
	dataset_name: maze.10-by-10-nondeterministic.simple
	sampler_name: probability
	sampler_kvargs: {}
	rollout_len: 10000
	rollout_repeats: 64
	prefix_len: 0
	min_reasoning_len: -1
	max_reasoning_len: -1


In [3]:
prompt_str = " ".join(rollout.prompt).replace("start", "\n\tstart").replace("goal", "\n\tgoal ").replace("wall", "\n\twall ")
response_str = " ".join(rollout.rollouts[0]).replace("bos ", "\n\tbos").replace("eos", "\n\teos").replace("create", "\n\tcreate").replace("close", "\n\tclose ").replace("plan ", "\n\tplan   ")

print("prompt: " + prompt_str)
print("response: " + response_str)

prompt: 
	start 3 6 
	goal  4 2 
	wall  0 0 
	wall  3 0 
	wall  4 0 
	wall  2 1 
	wall  4 1 
	wall  5 1 
	wall  9 1 
	wall  0 2 
	wall  1 2 
	wall  2 2 
	wall  6 2 
	wall  7 2 
	wall  5 3 
	wall  6 3 
	wall  7 3 
	wall  8 3 
	wall  9 3 
	wall  1 4 
	wall  2 4 
	wall  3 4 
	wall  9 4 
	wall  3 5 
	wall  4 5 
	wall  6 5 
	wall  5 6 
	wall  6 6 
	wall  9 6 
	wall  0 7 
	wall  2 7 
	wall  4 7 
	wall  6 8 
	wall  9 8 
	wall  2 9 
	wall  3 9 
	wall  4 9 
	wall  6 9 
	wall  8 9
response: 
	bos
	create 3 6 c0 c5 
	close  3 6 c0 c5 
	create 3 7 c1 c6 
	create 4 6 c1 c4 
	create 2 6 c1 c6 
	close  4 6 c1 c4 
	close  3 7 c1 c6 
	create 3 8 c2 c7 
	close  2 6 c1 c6 
	create 2 5 c2 c5 
	create 1 6 c2 c7 
	close  2 5 c2 c5 
	create 1 5 c3 c6 
	close  1 5 c3 c6 
	create 0 5 c4 c7 
	close  1 6 c2 c7 
	create 1 7 c3 c8 
	create 0 6 c3 c8 
	close  3 8 c2 c7 
	create 2 8 c3 c8 
	create 4 8 c3 c6 
	close  4 8 c3 c6 
	create 5 8 c4 c7 
	close  5 8 c4 c7 
	create 5 9 c5 c8 
	create 5 7 c5 c6 
	close  5 7 c5